# Template Notebook for NSGA II Multi-Objective-Optimization exploring the prompt embedding latent space

In [ ]:
from evolutionary.plotting import plot_fitness_statistics
from model_helpers.auto_pipeline import auto_diffusion_pipeline
from diffusers.utils import logging
from evolutionary.image_processing import create_animation_from_generations, create_generation_image_grid, save_images_from_generation

In [ ]:
logging.disable_progress_bar() # Or else your output will be full of progress bars
logging.set_verbosity_error() # Enable again if you are having problems

# Factory for loading the DiffusionPipeline from the HuggingFace Hub
def setup_pipeline():
    pipe = auto_diffusion_pipeline("stabilityai/sdxl-turbo") # This has to be compatible with the ImageCreator
    pipe.set_progress_bar_config(disable=True)
    return pipe

In [ ]:
from evolutionary.prompt_encoding.image_creation import SDXLPromptEmbeddingImageCreator
from evolutionary.prompt_encoding.variation import PooledArithmeticCrossover, PooledUniformGaussianMutator, UniformGaussianMutatorArguments
from evolutionary.selectors import TournamentSelector
from evolutionary.algorithms.nsga_ii import NSGA_II
from evolutionary.image_evaluators import AestheticsImageEvaluator, CLIPScoreEvaluator 
from evolutionary.evolution_base import MultiObjectiveEvaluator

prompt = "a cat sitting on a lamp in austria"
population_size = 100
num_generations = 100
batch_size = 1
elitism = 1

# Create the necessary components for the genetic algorithm
creator = SDXLPromptEmbeddingImageCreator(pipeline_factory=setup_pipeline, batch_size=batch_size, inference_steps=3)
evaluator = MultiObjectiveEvaluator([AestheticsImageEvaluator(), CLIPScoreEvaluator(prompt=prompt)])
crossover = PooledArithmeticCrossover(crossover_rate=0.5, crossover_rate_pooled=0.5)
# clamp_range was evaluated with pre-testing/clamp_range/sdxl_turbo.py
mutation_arguments = UniformGaussianMutatorArguments(mutation_rate=0.1, mutation_strength=2.5, clamp_range=(-900, 900)) 
mutation_arguments_pooled = UniformGaussianMutatorArguments(mutation_rate=0.1, mutation_strength=0.5, clamp_range=(-8, 8))
mutator = PooledUniformGaussianMutator(mutation_arguments, mutation_arguments_pooled)
selector = TournamentSelector(tournament_size=3)

# Prepare initial arguments
init_embed = creator.arguments_from_prompt(prompt)
init_args = [init_embed for _ in range(population_size)]

nsga = NSGA_II(
    num_generations=num_generations,
    population_size=population_size,
    solution_creator=creator,
    selector=selector,
    crossover=crossover,
    mutator=mutator,
    evaluator=evaluator,
    elitism_count=elitism,
    initial_arguments=init_args,
    post_evaluation_callback=lambda g, a: save_images_from_generation(a.population, g)  # TODO only best image save, change to post sort callback?
)

In [ ]:
best_solution = nsga.run()

In [ ]:
from diffusers.utils import make_image_grid

# Show best solution
print(best_solution.fitness)
make_image_grid(best_solution.result.images, 1, batch_size)

## Visualize the evolution

In [ ]:
for gen in range(num_generations):
    create_generation_image_grid(gen, max_images=10)

In [ ]:
video_loc = create_animation_from_generations(num_generations)
print(video_loc)
from IPython.display import Video
Video(filename=video_loc) # This does not work for all browsers/notebooks, set embed to true when you want to include it in the notebook (warning large file size)

## Plot fitness statistics

In [ ]:
plot_fitness_statistics(num_generations, nsga.best_fitness, nsga.worst_fitness, nsga.avg_fitness, title="Aesthetics", multi_objective_plot_index=0)

In [ ]:
plot_fitness_statistics(num_generations, nsga.best_fitness, nsga.worst_fitness, nsga.avg_fitness, title="CLIP Score", multi_objective_plot_index=1)

## Save notebook and components

In [ ]:
!jupyter nbconvert --to html nsga_notebook.ipynb

### Save the run to disk

In [ ]:
import pickle
import os
from datetime import datetime

os.makedirs("saved_runs", exist_ok=True)
with open(os.path.join("saved_runs", f"nsga_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.pkl"), "wb") as f:
    pickle.dump(nsga, f)

## Fallback functions for when something went wrong

### Access Best Solution from Disk

In [ ]:
import os
import glob
import evolutionary.image_processing as ip
from PIL import Image

num_generations = 4  # Set this to the number of generations you ran (if you didn't finish)
generation_dir = os.path.join("results", f"{num_generations}")
image_files = glob.glob(os.path.join(generation_dir, "*.png"))
image_files.sort(key=ip.fitness_filename_sorting_key, reverse=True)
print(image_files[0])
Image.open(image_files[0])

### ffmpeg is not installed, create GIF instead

In [ ]:
from evolutionary.image_processing import create_animation_from_generations_pil
video_loc = create_animation_from_generations_pil(num_generations)
print(video_loc)
from IPython.display import Video
Video(filename=video_loc) 